# 1. Build a convolutional neural network to classify object proposals (N+1 classes)

In [10]:
import json
import os
from sklearn.model_selection import train_test_split
from PIL import Image

# Load splits.json
with open('../Potholes/splits.json', 'r') as f:
    splits = json.load(f)

# Extract train and test splits
train_files = splits['train']
test_files = splits['test']

# Further split the train set into train and validation
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

In [11]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Paths
from Dataset import PotholeDataset
from tqdm import tqdm
img_dir = '../Potholes/annotated-images'
annotations_dir = '../Potholes/labeled_proposals'

# Initialize datasets
train_dataset = PotholeDataset(img_dir, annotations_dir, train_files, transform=transform)
val_dataset = PotholeDataset(img_dir, annotations_dir, val_files, transform=transform)
test_dataset = PotholeDataset(img_dir, annotations_dir, test_files, transform=transform)

# WeightedRandomSampler for training
labels = [label for _, label in tqdm(train_dataset,desc = "sampler for split")]

sampler for split: 100%|██████████| 142273/142273 [09:18<00:00, 254.87it/s]


In [ ]:
from OurDataloader import balanced_loader

class_bgd_ratio = 1/3
train_loader = balanced_loader(train_dataset, batch_size=16, ratio=class_bgd_ratio)  # DataLoader for training
val_loader = balanced_loader(val_dataset, batch_size=16, ratio=class_bgd_ratio)    # DataLoader for validation
# test_loader = balanced_loader(test_dataset, batch_size=16, ratio=class_bgd_ratio, shuffle=False)    # DataLoader for testing



separation of potholes:   8%|▊         | 2718/34684 [00:08<01:42, 312.10it/s]


KeyboardInterrupt: 

In [ ]:
import torch
from PotholeCNN import PotholeCNN
from TrainLoop import train
# Initialize model, optimizer, and load data
model = PotholeCNN()  # Your model class
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
train(model, optimizer, epochs=10, train_loader=train_loader, test_loader=val_loader, device=device)